# Extractor de Inteligencia Financiera Basado en Tweets

### **Objetivo**
Construir un script de Python que procese un dataset sintético de tweets financieros, extraiga información clave (entidades y sentimiento), estructure estos datos en un DataFrame limpio y realice un análisis para descubrir tendencias de sentimiento.

### **Paso 1: Configuración y Carga de Datos**

Primero, importa las librerías necesarias y carga el dataset de tweets `tweets.json`.

In [1]:
import pandas as pd
import os
os.environ["OMP_NUM_THREADS"] = "1"

import matplotlib 
import sklearn
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

import gensim
from gensim.models import Word2Vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import nltk


from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# Procesamiento
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder

# Métricas
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score

# Deep Learning
from keras.models import Sequential
from keras.layers import Dense, Input, Dropout
from keras.optimizers import SGD, RMSprop, Adam
from keras.losses import BinaryCrossentropy
from keras.metrics import BinaryAccuracy
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [2]:
df=pd.read_json("tweets.json")

In [3]:
df

,handler,tweet,likes,retweets,responses
0,@TechInvestor_Pro,"NVIDIA ($NVDA) is on another tear, hitting a n...",2371,473,121
1,@MarketPulseNews,Apple ($AAPL) investors getting a bit nervous ...,462,71,39
2,@satyanadella,Empowering developers is at the core of our mi...,22487,4513,1198
3,@sundarpichai,Excited to kick off Google I/O next week! We'l...,18194,3108,947
4,@amazon,We're continuing to invest in building the fut...,9788,1205,453
...,...,...,...,...,...
555,@Unity,We are listening to our developer community. W...,18811,4100,5100
556,@EpicGames,Unreal Engine 5 is empowering creators to buil...,22102,5100,1100
557,@FortniteGame,The new season is here. Drop in and discover a...,251022,61001,15000
558,@NvidiaGeforce,The new GeForce RTX 4080 SUPER is here. Up to ...,41022,8100,2500


### **Paso 2: Inicializar los Pipelines de NLP**

Necesitarás dos pipelines de Hugging Face para esta tarea: una pipeline de **Análisis de Sentimiento** y una pipeline de **Reconocimiento de Entidades Nombradas (NER)**.

In [4]:
from transformers import pipeline

sentiment_task = pipeline("text-classification", model="cardiffnlp/twitter-roberta-base-sentiment-latest")
sentiment_task("Covid cases are increasing fast!")

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'label': 'negative', 'score': 0.7235767245292664}]

In [5]:
NER = pipeline("token-classification", model="dslim/bert-base-NER",aggregation_strategy="simple")
example = """NVIDIA ($NVDA) is on another tear, hitting a new all-time high.
The demand for their AI chips seems insatiable. Jensen
Huang has built an absolute monster in Santa Clara. #AI #Investing #Semiconductors"""

ner_results = NER(df["tweet"][4])
print(ner_results)

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cpu


[{'entity_group': 'ORG', 'score': 0.9771534, 'word': 'Prime Health', 'start': 131, 'end': 143}]


### **Paso 3: Construir la Función Principal de Procesamiento**

Esta función procesará una única fila de tweet. Realizará un análisis de sentimiento, ejecutará NER para extraer entidades y luego devolverá una **lista de diccionarios**, donde cada diccionario representa una fila final y estructurada. Tienes que obtener una combinación única por cada persona, localidad y organización del tweet. Ejemplo:

- Tweet:
    ```text
    NVIDIA ($NVDA) is on another tear, hitting a new all-time high. The demand for their AI chips seems insatiable. Jensen Huang has built an absolute monster in Santa Clara. #AI #Investing #Semiconductors
    ```
- Entidades detectadas por la pipeline NER:
    - Personas: Jensen Huang
    - Orgnaizaciones: NVIDIA, NVDA
    - Localidades: Santa Clara

- Diccionario final:
    ```json
    [
        {
            "person": "Jensen Huang",
            "company": "NVIDIA",
            "location": "Santa Clara"
        },
        {
            "person": "Jensen Huang",
            "company": "NVDA",
            "location": "Santa Clara"
        }
    ]
    ```

In [6]:
    
#hago un analisis te sentimiento y devuelvo dos columnas una con el score y otra con el label
def analisis_sentimiento(columna):

    sent=sentiment_task(columna)
    score=sent[0]["score"]
    label=sent[0]["label"]

    return score,label

#ahora hago el ner y proceso  como se indica en el enunciado para dar una celda de lista de diccionarios inicial

def analisis_NER(columna):
    from itertools import product

    ner_results = NER(columna)

    dic={"person":[],"company":[],"location":[]}
    
    for elemento in ner_results:#para cada elmento de lo que devuelve el NER los clasifico en las 3 entidades para llenar el diccionario que se indica en el enunciado
        palabra=elemento["word"]
        tipoE=elemento["entity_group"]
        if "ORG" in tipoE and palabra not in dic["company"]:
            dic["company"].append(palabra)
        elif "PER" in tipoE and palabra not in dic["person"]:
            dic["person"].append(palabra)
        elif "LOC" in tipoE and palabra not in dic["location"]:
            dic["location"].append(palabra)

    dic_novacio={k:v if v else[None] for k,v in dic.items()}#con esto corrigo un error para considerar que uno de los 3 sea vacio
    
    
    dicc_formateado=[dict(zip(dic_novacio.keys(),values)) for values in product(*dic_novacio.values())]#con esta funcion doy todas las posibilidades con el producto cartesiano

    return dicc_formateado



#con esta funcion aplico las 2 anteriores a la columna que se indica llamando a lafuncion y devuelve el df_master para trabajar con el
def procesar_df(df,columna):

    df[["score_sentimiento","label_sentimiento"]]=df[columna].apply(analisis_sentimiento).apply(pd.Series)
    
    df["entidades"]=df[columna].apply(analisis_NER)

    #ademas utilizo la lista de diccionarios para ya sacar directamente las columnas con las que voy a trabajar

    df_expandido=df.explode("entidades").reset_index(drop=True)
    df_final=pd.concat([df_expandido.drop(columns=["entidades"]),pd.json_normalize(df_expandido["entidades"])],axis=1)


    return df_final


In [7]:
analisis_NER(df["tweet"][1])#aqui reviso que saca el formato del enunciado

[{'person': 'Tim Cook', 'company': 'Apple', 'location': 'Shanghai'},
 {'person': 'Tim Cook', 'company': 'SupplyC', 'location': 'Shanghai'}]

### **Paso 4: Ejecutar el Pipeline de Procesamiento**
Procesa todos los tweets y monta el DataFrame final.

In [8]:
df_master=procesar_df(df,"tweet")
#el procesamiento lo meti todo en una unica funcion para solo llamarla y ya ejecutar el procesado
df_master.head(20)


,handler,tweet,likes,retweets,responses,score_sentimiento,label_sentimiento,person,company,location
0,@TechInvestor_Pro,"NVIDIA ($NVDA) is on another tear, hitting a n...",2371,473,121,0.966121,positive,Jensen Huang,##VIDIA,Santa Clara
1,@MarketPulseNews,Apple ($AAPL) investors getting a bit nervous ...,462,71,39,0.532521,neutral,Tim Cook,Apple,Shanghai
2,@MarketPulseNews,Apple ($AAPL) investors getting a bit nervous ...,462,71,39,0.532521,neutral,Tim Cook,SupplyC,Shanghai
3,@satyanadella,Empowering developers is at the core of our mi...,22487,4513,1198,0.970738,positive,None,OpenAI,None
4,@sundarpichai,Excited to kick off Google I/O next week! We'l...,18194,3108,947,0.989660,positive,None,/,None
5,@amazon,We're continuing to invest in building the fut...,9788,1205,453,0.957007,positive,None,Prime Health,None
6,@Growth_To_Value,Meta ($META) stock is taking a beating after Z...,993,212,149,0.531419,negative,Z,Meta,Wall Street
7,@Growth_To_Value,Meta ($META) stock is taking a beating after Z...,993,212,149,0.531419,negative,Z,##uck,Wall Street
8,@Growth_To_Value,Meta ($META) stock is taking a beating after Z...,993,212,149,0.531419,negative,Z,Metaverse,Wall Street
9,@Growth_To_Value,Meta ($META) stock is taking a beating after Z...,993,212,149,0.531419,negative,Z,Metaver,Wall Street


### **Paso 5: Analizar los Datos Procesados**

Ahora que tienes un dataset estructurado, puedes extraer insights. Responde las siguientes preguntas usando el DataFrame `df_master`.

#### **Pregunta 1: ¿De quién y de qué se habla más?**
Identifica las 15 empresas y personas mencionadas con más frecuencia en el dataset.

In [9]:
df_master.groupby(["person"])["person"].value_counts().sort_values(ascending=False).head(15)

person
Jamie Dimon       5
Z                 4
Tim Cook          4
Warren Buffett    3
##ck Tan          2
Bob Iger          2
Sam Altman        2
Jensen Huang      2
Diamond           2
Bull              2
Ho                2
Blackwell         2
##lo              2
Marc Benioff      1
##erberg          1
Name: count, dtype: int64

In [10]:
df_master.groupby(["company"])["company"].value_counts().sort_values(ascending=False).head(15)

company
Apple          16
Fed            11
Amazon          8
Microsoft       6
Google          6
Tesla           6
Meta            5
Intel           4
Boeing          4
& P             4
Starbucks       4
Big Tech        4
Apple Store     4
##GR            4
Ford            3
Name: count, dtype: int64

In [11]:
df_master.groupby(["person","company"])[["person","company"]].value_counts().sort_values(ascending=False).head(15)

person          company           
Tim Cook        Apple                 3
Bull            Bank of America       2
##ck Tan        Broadcom              1
Jensen Huang    NVIDIA                1
Lisa Su         Intel                 1
M               ##R                   1
Mike Wilson     US Equity Strate      1
Sam Altman      AGI                   1
                OpenAI                1
Sun             Google                1
Tim Cook        SupplyC               1
Jamie Dimon     S                     1
Warren Buffett  Apple                 1
                Berkshire Hathaway    1
Z               ##uck                 1
Name: count, dtype: int64

### aqui dejo las 3 opciones al no estar seguro del enunciado,solo las 15 personas,solo las 15 empresas,juntandolos los 2

#### **Pregunta 2: ¿Cuál es la distribución general de sentimientos?**
Muestra el recuento de tweets POSITIVOS vs. NEGATIVOS en todo el dataset procesado.

In [12]:
df_master[df_master["label_sentimiento"].isin(["positive","negative"])].groupby(["label_sentimiento"])["label_sentimiento"].value_counts()

label_sentimiento
negative     53
positive    539
Name: count, dtype: int64

#### **Pregunta 3: ¿Qué empresas tienen el mayor y menor impacto de sentimiento?**
El sentimiento simple no es suficiente; el engagement (interacción) importa. Crea una puntuación de "impacto de sentimiento" que pondere el sentimiento por los 'likes' y 'retweets' del tweet. Un tweet positivo con muchos 'likes' debería tener un impacto mucho mayor que un tweet negativo con cero 'likes'.

Calcula esta puntuación y encuentra las 15 empresas con el mayor y menor impacto de sentimiento promedio.

In [13]:
df_master["impacto_sentimiento"]=df_master["score_sentimiento"] * (df_master["likes"] + df_master["retweets"])
df_master.head(3)#muestro la nueva columna de impactod e sentimiento

,handler,tweet,likes,retweets,responses,score_sentimiento,label_sentimiento,person,company,location,impacto_sentimiento
0,@TechInvestor_Pro,"NVIDIA ($NVDA) is on another tear, hitting a n...",2371,473,121,0.966121,positive,Jensen Huang,##VIDIA,Santa Clara,2747.648684
1,@MarketPulseNews,Apple ($AAPL) investors getting a bit nervous ...,462,71,39,0.532521,neutral,Tim Cook,Apple,Shanghai,283.833857
2,@MarketPulseNews,Apple ($AAPL) investors getting a bit nervous ...,462,71,39,0.532521,neutral,Tim Cook,SupplyC,Shanghai,283.833857


### 15 empresas con mas media de impacto

In [14]:
df_master.groupby(["company"])["impacto_sentimiento"].mean().sort_values(ascending=False).head(15)

company
Stars              695890.128793
Spice Latte        337690.807015
##R                198514.072308
Op                 197932.600966
##mus              197932.600966
P                  173682.117231
MacBookAir         134157.278531
MacBook Air        134157.278531
Starlink           122246.874797
WWDC24             102151.255389
##C                102151.255389
BlueO               97004.155391
Community Notes     91410.902470
Apple Store         85173.807341
Artemis             61477.303101
Name: impacto_sentimiento, dtype: float64

### 15 empresas con menos media de impacto

In [15]:
df_master.groupby(["company"])["impacto_sentimiento"].mean().sort_values(ascending=False).tail(15)

company
JPMorgan              310.398524
Delta Air Lines       306.607343
SupplyC               283.833857
Temu                  283.063345
Costco                275.741777
Kirkland              275.741777
Ho                    232.310190
On                    232.310190
Nike                  232.310190
Bank of America       229.302970
In                    218.700172
Adobe                 206.402699
Disney                191.251263
UnitedHealth Group    188.856611
UNH                   188.856611
Name: impacto_sentimiento, dtype: float64

#### **Pregunta 4: ¿Qué individuos tienen el mayor y menor impacto de sentimiento?**
Usando la misma puntuación de "impacto de sentimiento", encuentra los 15 individuos con el mayor y menor impacto de sentimiento promedio.

### las 15 personas con mas media de impacto

In [16]:
df_master.groupby(["person"])["impacto_sentimiento"].mean().sort_values(ascending=False).head(15)

person
Travis Scott    275983.083072
M               198514.072308
Diab            101060.339129
G                97004.155391
Fe               97004.155391
##lo             57277.102763
Que              54802.830284
##so Blanco      54802.830284
Chip             45976.283473
Sam Altman       15961.866676
##erberg         15841.328647
Zuck             15841.328647
F                13493.866398
Nelson Peltz     12561.759449
Benio            10857.808025
Name: impacto_sentimiento, dtype: float64

### las 15 personas con menos media de impacto

In [17]:
df_master.groupby(["person"])["impacto_sentimiento"].mean().sort_values(ascending=False).tail(15)

person
Diamond           2061.021955
Warren Buffett    1232.677042
##ck Tan          1191.050488
Ho                1191.050488
Alex Karp          991.680715
Lisa Su            981.740806
Z                  640.359654
Disney             597.764501
Tim Cook           501.291536
##dar Picha        294.401888
Sun                294.401888
Jeff Bezos         275.536991
Andy Jassy         275.536991
Marc Benioff       265.361408
Bull               229.302970
Name: impacto_sentimiento, dtype: float64